In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Step 1: Fetch the webpage
url = "https://www.fundamentus.com.br/resultado.php"

header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

response = requests.get(url, headers=header)
response.encoding = 'utf-8'  # Ensure proper encoding

# Step 2: Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')  # Locate the table

# Step 3: Convert HTML table to pandas DataFrame
df = pd.read_html(str(table))[0]

# Step 4: Save DataFrame to CSV
df.to_csv('fundamentus_data.csv', index=False)


In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

# LLM Chain

In [18]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# Load the CSV data
csv_tool = CSVLoader("fundamentus_data.csv")


In [25]:
# We can simply question LLM if we want
# llm.invoke("how can langsmith help with testing?")

system_prompt = """
You are a world-class investor with expertise in value investing, akin to Joel Greenblatt, known for your ability to identify undervalued companies with high potential returns. Your investment strategy combines a deep understanding of financial statements, a rigorous approach to calculating intrinsic value, and a focus on special situations such as spinoffs, mergers, and restructurings. You prioritize long-term value over short-term gains and are skilled at identifying businesses with strong competitive advantages and solid management.

When providing investment advice or answering questions, focus on fundamental analysis, including key metrics such as Return on Capital (ROC), Earnings Yield, and Price-to-Earnings (P/E) ratios. Your responses should consider market conditions, industry trends, and the potential risks associated with each investment. You also emphasize the importance of a margin of safety in every investment decision, ensuring that investments are made at a significant discount to their intrinsic value.

Your goal is to provide precise, well-reasoned, and insightful responses that reflect the principles of value investing and the mindset of a disciplined, patient investor who seeks to maximize returns while minimizing risk.
"""

# Or we can guide LLM response with a prompt template as follows:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    ("agent", "ai")  # Add this line   
])

# combine these into a simple LLM chain
# chain = prompt | llm



ValueError: Unexpected message type: agent. Use one of 'human', 'user', 'ai', 'assistant', or 'system'.

In [22]:
from langchain.agents import create_openai_functions_agent



# Step 3: Create an agent that can utilize both the prompt and the CSV data
agent = create_openai_functions_agent(
    llm=llm,
    tools=[csv_tool],  # Adding the CSVLoaderTool
    prompt=prompt
)

# Step 4: Interact with the agent
query = "Which company from the data has the highest return on capital?"
response = agent.run(query)
print(response)

# chain.invoke({"input": "create a rank with top 10 companies accorindly to the magic formula"})

ValueError: Prompt must have input variable `agent_scratchpad`, but wasn't found. Found ['input'] instead.